In [2]:
# imports
import pandas as pd   # to process data
import numpy as np
import requests     # to fetch the data from the api

In [3]:
# define common variables

#  link is the api link which will return the data
# q is the query keyword to search for .i.e apple and microsoft
# end and start date will be used to specify the range
link = "http://api.nytimes.com/svc/search/v2/articlesearch.json?q={}&begin_date={}&end_date={}&api-key=ce622601a6b047dc88b57aff2a04e921"
final_link=''
blank_count=0    # var to count the number of blank returns fromt he api

In [4]:
# for the microsoft

In [5]:
# read the csv files to have a basic data frame to work with
micronews= pd.read_csv('MSFT.csv')   # a data frame to add news
micronews['News']='' # add a column to store the news 

microsenti= pd.read_csv('MSFT.csv')  # data frame to store the sentiment analysis
microsenti["compound"] = float      # extra columns to store the sentiment score
microsenti["neg"] = float
microsenti["neu"] = float
microsenti["pos"] = float

In [ ]:
blank_count=0
for row in range(0,len(micronews)):       # change the range values to refetch the data specific fields
    # get the date to fetch the data from the data frame
    day=str(micronews['Date'][row].replace('-',''))
    # add this data into the final link
    final_link= link.format('microsoft',day,str(int(day)+1))
    # get the data from the api
    r = requests.get(final_link)
    data = r.json()  
    
    news=''   # blank string
    # if there is news on that given date range than the api return will contain the 'response' as one of the key
    if 'response' in data.keys():
        for i in range(len(data['response']['docs'])):
            news+=str(data['response']['docs'][i]['headline']['main'])+' : '+str(data['response']['docs'][i]['snippet'])
        micronews['News'][row]=news    
        blank_count = 0
    
    # for the case when there is no data about that that topic
    else:
        print(row,data)
        blank_count += 1
    # if it doesnot give data for 50 contineous dates, than this implies that there is some problem with the api or the api key
    if blank_count >= 50: 
        print(row,data)
        break

In [ ]:
# write the data into a csv file
micronews.to_csv('MicrosoftNews.csv')

In [ ]:
# now use the abone news dataframe to find its sentiment score
# few extra headers
# from nltk.classify import NaiveBayesClassifier
# from nltk.corpus import subjectivity
# from nltk.sentiment import SentimentAnalyzer
# from nltk.sentiment.util import *

import nltk
nltk.download('vader_lexicon')

from nltk.sentiment.vader import SentimentIntensityAnalyzer
import unicodedata

sid = SentimentIntensityAnalyzer()
for date, row in micronews.T.iteritems():
    try:
        ss = sid.polarity_scores( micronews.loc[date, 'News'])
        microsenti.set_value(date, 'compound', ss['compound'])
        microsenti.set_value(date, 'neg', ss['neg'])
        microsenti.set_value(date, 'neu', ss['neu'])
        microsenti.set_value(date, 'pos', ss['pos'])
    except TypeError:
        print (micronews.loc[date, 'articles'])
        print (date)


In [ ]:
# final data processing
microsenti=microsenti.round(4)    
microsenti=microsenti.drop(['Volume'],axis=1)    # remove the unused field

In [ ]:
# store the final data set created
microsenti.to_csv('MicrosoftFinalData.csv')

# this complete process will be perfomed with the apple data set also